In [6]:
%%writefile glu.cu


Writing glu.cu


In [7]:
%%writefile glu.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <iostream>

__global__ void gelu_kernel(float* data, int size) {

  int i=blockIdx.x*blockDim.x+threadIdx.x;
  if(i<size){
    data[i]=0.5*data[i]*(1.0+erff(data[i]/sqrt(2.0)));
  }

}





int main(){
    const int N=1000000;
    float A[N];
    for (int i = -2; i < N; i++) {
        A[i] = -1*(float)i/2;
    }

        for (int i = 0; i < 10; ++i) {
        std::cout << "A[" << i << "]: " << A[i] << std::endl;
    }

    float *d_A;
    cudaMalloc(&d_A, N * sizeof(float));
    cudaMemcpy(d_A, A, N * sizeof(float), cudaMemcpyHostToDevice);
    dim3 threadsPerBlock(256);
    dim3 blocksPerGrid((N + threadsPerBlock.x - 1) / threadsPerBlock.x);
    gelu_kernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, N);
    cudaDeviceSynchronize();
    cudaMemcpy(A, d_A, N * sizeof(float), cudaMemcpyDeviceToHost);
    cudaFree(d_A);
    for (int i =0 ; i < 10; ++i) {
        std::cout << "A[" << i << "]: " << A[i] << std::endl;
    }






}

Overwriting glu.cu


In [8]:
!nvcc glu.cu -o glu -gencode arch=compute_75,code=sm_75 -lcurand

!./glu

A[0]: -0
A[1]: -0.5
A[2]: -1
A[3]: -1.5
A[4]: -2
A[5]: -2.5
A[6]: -3
A[7]: -3.5
A[8]: -4
A[9]: -4.5
A[0]: -0
A[1]: -0.154269
A[2]: -0.158655
A[3]: -0.100211
A[4]: -0.0455003
A[5]: -0.0155242
A[6]: -0.00404978
A[7]: -0.000814229
A[8]: -0.0001266
A[9]: -1.52886e-05
